Historical VaR

Even though the past is not an indication of future performance, this is still a insightful indication of downside to bear in mind.

Historical VaR consists in calculating the nth worst outcome out of the historical sample. Below you can see one possible way to calculate it in Python:

In [8]:
import pandas as pd
import yfinance as yf
def download_ticker(ticker):
    data_= yf.download(tickers = ticker,
                        start="2017-01-01",
                        #end="2017-12-31",
                        interval = "1d",
                        auto_ajust = True,
                        threads = True,
                           )
    #display(data_["Adj Close"])
    return data_["Adj Close"]

In [9]:
download_ticker("AAPL")

[*********************100%***********************]  1 of 1 completed


Date
2017-01-03     27.413372
2017-01-04     27.382690
2017-01-05     27.521944
2017-01-06     27.828764
2017-01-09     28.083660
                 ...    
2021-07-01    137.270004
2021-07-02    139.960007
2021-07-06    142.020004
2021-07-07    144.570007
2021-07-08    143.240005
Name: Adj Close, Length: 1136, dtype: float64

In [11]:
import numpy as np
def var_historic(r, level=1):
    """
    Takes in a series of returns (r), and the percentage level
(level)
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        return -np.percentile(r, level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [13]:
var_historic(download_ticker("AAPL").pct_change().dropna())

[*********************100%***********************]  1 of 1 completed


0.052525469860639905

Parametric and Semi-Parametric VaR
Another way to calculate VaR is to assume the set of possible outcomes behaves like a Normal (Gaussian) Distribution
The main drawback of the parametric approach is that real world returns usually have a distribution with “fat tails” (high kurtosis).
<img src="https://miro.medium.com/max/700/1*tjgMEXMLizt-48DLr1Gwrw.png" >
To illustrate how this is important, let’s say that you calculate the VAR for 1% using a parametric approach and it comes up as 6%. Then, you look at real historical data, and you see that a -6% return happens way more often then 1% of the time: this would be an indication of fat-tails.

Below you will find some code for parametric and semiparametric VaR

In [15]:
from scipy.stats import norm
def var_gaussian(r, level=5, modified=False):
    """
    Returns the Parametric Gauuian VaR of a Series or DataFrame
    If "modified" is True, then the modified VaR is returned,
    using the Cornish-Fisher modification
    """
    # compute the Z score assuming it was Gaussian
    z = norm.ppf(level/100)
    if modified:
        # modify the Z score based on observed skewness and kurtosis
        s = skewness(r)
        k = kurtosis(r)
        z = (z +
                (z**2 - 1)*s/6 +
                (z**3 -3*z)*(k-3)/24 -
                (2*z**3 - 5*z)*(s**2)/36
            )
    return -(r.mean() + z*r.std(ddof=0))

In [16]:
var_gaussian(download_ticker("AAPL").pct_change().dropna())

[*********************100%***********************]  1 of 1 completed


0.030667009789832063

Lastly, another method for calculating VaR consists of simulating possible random outcomes and using that as the distribution for the calculation.

# VaR with Monte Carlo